---


<h1 style="text-align:center"> 
Lung Cancer Classification using CT Data (Smoke Detector)
</h1><style>

--- 
## Libraries


In [1]:
import pandas as pd
import pylidc as pl
import xml.etree.ElementTree as ET
import matplotlib as plt
import numpy as np

---

## Importing XML files

In [ ]:
plt.set_cmap('gray')

In [6]:
#%pip install xlrd

In [ ]:
nodule_counts_df = pd.read_excel('lidc-idri-nodule-counts-6-23-2015.xlsx')
nodule_counts_df
#maybe eliminar quarta e quinta coluna

In [ ]:
pacient_data_df = pd.read_excel('tcia-diagnosis-data-2012-04-20.xls')
pacient_data_df 
#nem todos os pacientes têm historial registado (157 em 1010)

---


## Importing Annotations

In [ ]:
annotation = pl.query(pl.Annotation).first()
print(annotation.scan.patient_id)

In [ ]:
pid = 'LIDC-IDRI-0078'
scan = pl.query(pl.Scan).filter(pl.Scan.patient_id == pid).first()
print(len(scan.annotations))

In [ ]:
nods = scan.cluster_annotations()

In [ ]:
annotation.visualize_in_3d()

### Visualization of the annotated nodule

In [ ]:
volume = annotation.scan.to_volume()
padding = [(30,10), (10,25), (0,0)]

mask = annotation.boolean_mask(pad=padding)
bbox = annotation.bbox(pad=padding)

fig,ax = plt.subplots(1,2,figsize=(5,3))

ax[0].imshow(vol[bbox][:,:,2], cmap=plt.cm.gray)
ax[0].axis('off')

ax[1].imshow(mask[:,:,2], cmap=plt.cm.gray)
ax[1].axis('off')

plt.tight_layout()
plt.savefig("../images/mask_bbox.png", bbox_inches="tight")
plt.show()

### Surface Visualization

In [ ]:

# Verifica os colormaps disponíveis
print(plt.colormaps())

# Carregar a anotação de lobulação 5
ann = pl.query(pl.Annotation)\
        .filter(pl.Annotation.lobulation == 5).first()

# Usar a função visualize_in_3d com colormap válido
ann.visualize_in_3d(cmap='viridis') 

---

In [4]:
path_to_annotations = 'LIDC-XML-only'

In [6]:
path = 'C:/Users/Daniela/Desktop/Fac/3ºano/Lab_IACD/Project1/manifest-1727162347556/LIDC-IDRI'
f = open('C:/Users/Daniela/pylidc.conf', 'w')
f.write(f'[dicom]\npath = {path}\n\n')
f.close()

In [3]:
pid = 'LIDC-IDRI-0001'
scan = pl.query(pl.Scan).filter(pl.Scan.patient_id == pid).first()

In [ ]:
vol = scan.to_volume()
print(vol.shape) # (dim, dim, depth)

plt.figure(figsize=(5, 5))
plt.imshow(vol[:,:,0])
plt.show() # show the first-level slice

In [ ]:
#extract information from an XML file
def parse_xml(file):
    tree = ET.parse(file)
    root = tree.getroot()

    data = []
    for nodule in root